In [1]:
# Import the dependencies.
import pandas as pd
import numpy as np
from citipy import citipy

# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [3]:
# Add the latititudes and longitutes to a list. 
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

734

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [6]:
city_url = url + "appid" + weather_api_key + "&q=" + city

In [7]:
weather_response = requests.get(city_url)
weather_json = weather_response.json()

In [8]:
weather_json.keys()

dict_keys(['cod', 'message'])

In [20]:
# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [21]:
# Loop through all the cities in our list
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]       
            
            # Append the City information into city_data list
        city_data.append({"City": city.title(), 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})
    # If an error is experienced, skip the city

    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | torbay
Processing Record 2 of Set 1 | mehamn
Processing Record 3 of Set 1 | yellowknife
Processing Record 4 of Set 1 | ahipara
Processing Record 5 of Set 1 | oktyabrskoye
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | francisco beltrao
Processing Record 8 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 9 of Set 1 | hilo
Processing Record 10 of Set 1 | suleja
Processing Record 11 of Set 1 | puerto ayora
Processing Record 12 of Set 1 | busselton
Processing Record 13 of Set 1 | hobart
Processing Record 14 of Set 1 | saskylakh
Processing Record 15 of Set 1 | bengkulu
Processing Record 16 of Set 1 | port alfred
Processing Record 17 of Set 1 | airai
Processing Record 18 of Set 1 | rocha
Processing Record 19 of Set 1 | atasu
Processing Record 20 of Set 1 | bluff
Processing Record 21 of Set 1 | samarai
Processing Record 22 of Set 1 | pevek
Processing Record 23 of Set 1 | mataura
Processing Record 24 of Set 1 | sa

In [22]:
len(city_data)

681

In [23]:
# Convert the array of dictionaries to a Pandas dataframe. 
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,Torbay,47.6666,-52.7314,45.39,scattered clouds,64,40,25.32,CA
1,Mehamn,71.0357,27.8492,37.33,overcast clouds,89,100,29.53,NO
2,Yellowknife,62.4560,-114.3525,14.52,broken clouds,48,75,9.22,CA
3,Ahipara,-35.1667,173.1667,68.00,overcast clouds,69,100,17.34,NZ
4,Oktyabrskoye,43.0645,44.7417,32.14,clear sky,100,0,8.95,RU
5,Punta Arenas,-53.1500,-70.9167,46.51,scattered clouds,81,40,13.80,CL
6,Francisco Beltrao,-26.0811,-53.0550,52.99,clear sky,84,0,3.31,BR
7,Hilo,19.7297,-155.0900,73.27,heavy intensity rain,92,100,4.61,US
8,Suleja,9.1758,7.1808,81.59,light rain,73,95,2.17,NG
9,Puerto Ayora,-0.7393,-90.3518,75.40,broken clouds,91,79,5.32,EC


In [24]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
cit_data_df = city_data_df[new_column_order]
cit_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Torbay,CA,47.6666,-52.7314,45.39,64,40,25.32,scattered clouds
1,Mehamn,NO,71.0357,27.8492,37.33,89,100,29.53,overcast clouds
2,Yellowknife,CA,62.4560,-114.3525,14.52,48,75,9.22,broken clouds
3,Ahipara,NZ,-35.1667,173.1667,68.00,69,100,17.34,overcast clouds
4,Oktyabrskoye,RU,43.0645,44.7417,32.14,100,0,8.95,clear sky
5,Punta Arenas,CL,-53.1500,-70.9167,46.51,81,40,13.80,scattered clouds
6,Francisco Beltrao,BR,-26.0811,-53.0550,52.99,84,0,3.31,clear sky
7,Hilo,US,19.7297,-155.0900,73.27,92,100,4.61,heavy intensity rain
8,Suleja,NG,9.1758,7.1808,81.59,73,95,2.17,light rain
9,Puerto Ayora,EC,-0.7393,-90.3518,75.40,91,79,5.32,broken clouds


In [25]:
# Create the output file (CSV).
output_data_file = "weather_database/WeatherPY_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")